In [ ]:
# @title Install necessary libraries
!pip install openai ipywidgets graphviz ipython --quiet

# @title Load OpenAI API Credentials
from getpass import getpass
import os
import openai

# Check if key is already set (e.g., in Colab secrets)
api_key_env = os.environ.get('OPENAI_API_KEY')

if api_key_env:
    print("OpenAI API Key found in environment variables.")
    OPENAI_KEY = api_key_env
else:
    print("OpenAI API Key not found in environment variables.")
    # Fallback to getpass if not found
    try:
        OPENAI_KEY = getpass('Enter your OpenAI API Key: ')
        os.environ['OPENAI_API_KEY'] = OPENAI_KEY
        openai.api_key = OPENAI_KEY
        # Test the key with a simple call (optional but recommended)
        # client = openai.OpenAI()
        # client.models.list()
        print("OpenAI API Key configured.")
    except Exception as e:
        print(f"Failed to configure OpenAI API Key: {e}")
        OPENAI_KEY = None # Ensure it's None if setup failed

# Configure the OpenAI client (using the newer openai > 1.0 library structure)
if OPENAI_KEY:
    try:
      client = openai.OpenAI(api_key=OPENAI_KEY)
      print("OpenAI client initialized successfully.")
    except Exception as e:
      print(f"Error initializing OpenAI client: {e}")
      client = None
else:
    print("Cannot initialize OpenAI client: API Key is missing.")

# @title Import other required libraries
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import graphviz
import uuid # For generating unique node IDs
import json # For potentially storing/loading graphs
import time # For simulating delays if needed
import traceback # For better error reporting
import copy # For deep copying node data if needed

In [ ]:
# @title Node Graph Data Structure and LLM Function

# Global dictionary to store our nodes and their connections
# Structure Change: 'outputs' values are now LISTS of node IDs
# nodes = {
#     'node_id_1': {
#         'name': 'User Friendly Name', 'type': 'Button', 'options': {},
#         'outputs': {'on_click': ['node_id_2', 'node_id_4']} # <<< LIST HERE
#     },
#     'node_id_2': {
#         'name': 'Ask LLM', 'type': 'LLM Program', 'options': {'prompt': '...'},
#         'outputs': {'result': ['node_id_3']} # <<< LIST HERE
#      },
#      'node_id_3': {
#         'name': 'Show Result', 'type': 'Show', 'options': {'text_template': '...'},
#         'outputs': {} # No outputs from Show node
#      },
#      'node_id_4': {
#          'name': 'Show Another', 'type': 'Show', 'options': {'text_template': 'Button also triggered this!'},
#          'outputs': {}
#      }
# }
nodes = {}
editing_node_id = None # Track which node ID is currently being edited

# --- LLM Interaction Function (No Change) ---
def call_llm(prompt_text, model="gpt-3.5-turbo"):
    """Calls the OpenAI API to get a completion."""
    if not client:
        return "Error: OpenAI client not initialized. Please check API Key."
    try:
        # print(f"\nDEBUG: Calling LLM with prompt: '{prompt_text[:100]}...'") # Debug print
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant executing user instructions."},
                {"role": "user", "content": prompt_text}
            ]
        )
        result = response.choices[0].message.content.strip()
        # print(f"DEBUG: LLM Response received: '{result[:100]}...'") # Debug print
        return result
    except Exception as e:
        error_message = f"Error calling OpenAI API: {e}"
        print(f"DEBUG: {error_message}") # Debug print
        return f"Error: Could not get response from LLM. Details: {e}"

print("Node Graph Data Structure (with list outputs) and LLM function defined.")

In [ ]:
# @title Graph Visualization Function (Handles Multiple Connections)

graph_output = widgets.Output()

def visualize_graph():
    """Renders the current node graph using Graphviz, handling multiple outputs."""
    dot = graphviz.Digraph(comment='Node Graph', format='png')
    dot.attr(rankdir='LR')

    with dot.subgraph(name='cluster_nodes') as c:
        c.attr(label='Nodes', style='filled', color='lightgrey')
        for node_id, node_data in nodes.items():
            node_label = f"ID: {node_id}\nType: {node_data['type']}\nName: {node_data['name']}"
            # Add options preview if applicable
            if node_data['type'] == 'LLM Program':
                node_label += f"\nPrompt: {node_data['options'].get('prompt', '')[:30]}..."
            elif node_data['type'] == 'Show':
                 node_label += f"\nText: {node_data['options'].get('text_template', '')[:30]}..."

            # Highlight node being edited
            node_color = 'lightblue' if node_id == editing_node_id else 'white'
            c.node(node_id, label=node_label, shape='box', style='filled', fillcolor=node_color)

    # Add edges after all nodes are defined
    for node_id, node_data in nodes.items():
        for port_name, target_node_ids in node_data.get('outputs', {}).items():
            # Ensure target_node_ids is a list (even if loaded from old format)
            if isinstance(target_node_ids, str):
                target_node_ids = [target_node_ids] # Convert old format on the fly

            if isinstance(target_node_ids, list):
                for target_node_id in target_node_ids: # Iterate through the list
                    if target_node_id in nodes:
                        dot.edge(node_id, target_node_id, label=port_name)
                    else:
                        print(f"Warning: Connection from {node_id} ({port_name}) points to non-existent node {target_node_id}")
            # else: Handle potential data corruption? print warning?

    with graph_output:
        clear_output(wait=True)
        if not nodes:
            print("Graph is empty. Add some nodes!")
        else:
            try:
                display(dot)
            except graphviz.backend.execute.ExecutableNotFound:
                 display(Markdown("**Error:** `graphviz` executable not found. Rendering failed."))
            except Exception as e:
                 display(Markdown(f"**Error rendering graph:** {e}"))

print("Graph Visualization function updated for multiple connections.")
# Initial visualization (will be empty)
visualize_graph()

In [ ]:
# @title Node Definition, Editing, and Removal UI (MODIFIED for Text Input)

import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import uuid
import graphviz # Ensure graphviz is imported if not already globally

# --- Global State ---
if 'nodes' not in globals(): nodes = {}
editing_node_id = None # Track which node ID is currently being edited

# --- Output Widgets ---
edit_feedback_output = widgets.Output()
add_node_output = widgets.Output() # Combined output for add/update actions
add_connection_output = widgets.Output()
if 'graph_output' not in globals(): graph_output = widgets.Output()
if 'visualize_graph' not in globals():
    def visualize_graph(): print("Error: visualize_graph not defined") # Placeholder

# --- VALID NODE TYPES --- Define consistently
VALID_NODE_TYPES = ['Button', 'LLM Program', 'Show', 'Text Input'] # Added 'Text Input'
NODE_TYPE_TO_INDEX = {t: i for i, t in enumerate(VALID_NODE_TYPES)}
INDEX_TO_NODE_TYPE = {i: t for i, t in enumerate(VALID_NODE_TYPES)}

# --- Helper: Get Node Choices for Dropdowns ---
def get_node_choices():
    """Returns a dictionary of 'Display Name': node_id for dropdowns."""
    choices = {f"{data['name']} ({node_id})": node_id for node_id, data in nodes.items()}
    if not choices:
        return {"(No nodes available)": None}
    return choices

# --- Helper Functions for Dropdowns and Ports ---
def update_all_dropdowns():
    """Updates all node selection dropdowns."""
    global source_node_dropdown, target_node_dropdown, select_node_edit_dropdown # Make them accessible

    node_choices = get_node_choices()
    current_source = source_node_dropdown.value
    current_target = target_node_dropdown.value
    current_edit_selection = select_node_edit_dropdown.value

    # Update options
    source_node_dropdown.options = node_choices
    target_node_dropdown.options = node_choices
    select_node_edit_dropdown.options = node_choices

    # Try to restore selections
    source_node_dropdown.value = current_source if current_source in node_choices.values() else (list(node_choices.values())[0] if node_choices else None)
    target_node_dropdown.value = current_target if current_target in node_choices.values() else (list(node_choices.values())[0] if node_choices else None)
    select_node_edit_dropdown.value = current_edit_selection if current_edit_selection in node_choices.values() else (list(node_choices.values())[0] if node_choices else None)

    update_source_ports(None) # Update ports based on potentially changed source node

def update_source_ports(change):
    """Update the source port dropdown based on the selected source node."""
    if 'source_port_dropdown' not in globals(): return # Widget not ready
    selected_node_id = source_node_dropdown.value
    port_options = []
    if selected_node_id and selected_node_id in nodes:
        node_type = nodes[selected_node_id]['type']
        if node_type == 'Button': port_options = ['on_click']
        elif node_type == 'LLM Program': port_options = ['result']
        elif node_type == 'Text Input': port_options = ['text_out'] # Added output for Text Input
        # 'Show' nodes have no output ports in this model
    source_port_dropdown.options = port_options
    source_port_dropdown.value = port_options[0] if port_options else None


# --- Widgets for Node Creation/Editing ---
node_type_dropdown = widgets.Dropdown(options=VALID_NODE_TYPES, value='Button', description='Node Type:', style={'description_width': 'initial'}) # Use VALID_NODE_TYPES
node_name_text = widgets.Text(value='My Node', placeholder='Enter a descriptive name', description='Node Name:', style={'description_width': 'initial'})

# Option Boxes
button_options_box = widgets.VBox([])
llm_options_box = widgets.VBox([widgets.Textarea(value='Summarize the input.', placeholder='Enter LLM prompt...', description='LLM Prompt:', layout={'width': '95%'}, style={'description_width': 'initial'})])
show_options_box = widgets.VBox([widgets.Textarea(value='{input}', placeholder='Enter text. Use {input} for data...', description='Show Text:', layout={'width': '95%'}, style={'description_width': 'initial'})])
text_input_options_box = widgets.VBox([widgets.Textarea(value='Enter your static text here.', placeholder='Enter content...', description='Text Content:', layout={'width': '95%', 'height': '100px'}, style={'description_width': 'initial'})]) # Added for Text Input

# Accordion including the new options box
options_accordion = widgets.Accordion(
    children=[button_options_box, llm_options_box, show_options_box, text_input_options_box], # Added text_input_options_box
    selected_index=0
)
# Set titles dynamically based on VALID_NODE_TYPES order
for i, node_type in enumerate(VALID_NODE_TYPES):
    options_accordion.set_title(i, f'{node_type} Options')


add_update_node_button = widgets.Button(description='Add Node', button_style='success', icon='plus')
cancel_edit_button = widgets.Button(description='Cancel Edit', button_style='warning', icon='times', visible=False) # Initially hidden

# --- Widgets for Node Selection & Management ---
select_node_edit_dropdown = widgets.Dropdown(description='Select Node:', style={'description_width': 'initial'})
load_node_button = widgets.Button(description='Load for Edit', button_style='info', icon='edit')
remove_node_button = widgets.Button(description='Remove Selected Node', button_style='danger', icon='trash')

# --- Widgets for Adding Connections ---
source_node_dropdown = widgets.Dropdown(description='Source Node:', style={'description_width': 'initial'})
source_port_dropdown = widgets.Dropdown(description='Output Port:', style={'description_width': 'initial'})
target_node_dropdown = widgets.Dropdown(description='Target Node:', style={'description_width': 'initial'})
add_connection_button = widgets.Button(description='Add Connection', button_style='primary', icon='link')


# --- Event Handlers ---

def reset_node_form(clear_name=True):
    """Resets the node creation form to default."""
    global editing_node_id
    editing_node_id = None
    node_type_dropdown.value = 'Button' # Default to Button
    if clear_name: node_name_text.value = 'My Button' # Reset name
    # Reset options for all types
    llm_ta = next((w for w in llm_options_box.children if isinstance(w, widgets.Textarea)), None)
    if llm_ta: llm_ta.value = 'Summarize the input.'
    show_ta = next((w for w in show_options_box.children if isinstance(w, widgets.Textarea)), None)
    if show_ta: show_ta.value = '{input}'
    text_input_ta = next((w for w in text_input_options_box.children if isinstance(w, widgets.Textarea)), None)
    if text_input_ta: text_input_ta.value = '' # Clear text input content

    options_accordion.selected_index = NODE_TYPE_TO_INDEX['Button'] # Reset accordion
    add_update_node_button.description = 'Add Node'
    add_update_node_button.button_style = 'success'
    add_update_node_button.icon = 'plus'
    cancel_edit_button.visible = False
    node_type_dropdown.disabled = False # Re-enable type change for adding
    visualize_graph() # Redraw graph to remove edit highlight

def on_cancel_edit_clicked(b):
    """Resets the form when Cancel Edit is clicked."""
    with add_node_output:
        clear_output(wait=True)
        print("Edit cancelled.")
    reset_node_form()

def handle_node_type_change(change):
    """Sync accordion and suggest name on type change, only if NOT editing."""
    if editing_node_id is None: # Only act if we are adding a new node
        node_type = change.new
        options_accordion.selected_index = NODE_TYPE_TO_INDEX.get(node_type, 0)
        # Suggest default names
        if node_type == 'LLM Program': node_name_text.value = 'Ask LLM'
        elif node_type == 'Show': node_name_text.value = 'Display Result'
        elif node_type == 'Button': node_name_text.value = 'Start Button'
        elif node_type == 'Text Input': node_name_text.value = 'My Text' # Added suggestion


node_type_dropdown.observe(handle_node_type_change, names='value') # Observe AFTER function defined

def on_add_update_node_clicked(b):
    """Adds a new node or updates the currently editing node."""
    global editing_node_id
    node_type = node_type_dropdown.value
    node_name = node_name_text.value.strip() or f"Untitled_{node_type}"

    if not node_name:
         with add_node_output: clear_output(wait=True); print("❌ Error: Node name cannot be empty.")
         return

    options = {}
    # Extract options based on type
    if node_type == 'LLM Program':
        llm_ta = next((w for w in llm_options_box.children if isinstance(w, widgets.Textarea)), None)
        if llm_ta: options['prompt'] = llm_ta.value
    elif node_type == 'Show':
        show_ta = next((w for w in show_options_box.children if isinstance(w, widgets.Textarea)), None)
        if show_ta: options['text_template'] = show_ta.value
    elif node_type == 'Text Input': # Added option extraction
        text_input_ta = next((w for w in text_input_options_box.children if isinstance(w, widgets.Textarea)), None)
        if text_input_ta: options['content'] = text_input_ta.value

    with add_node_output:
        clear_output(wait=True)
        if editing_node_id: # --- UPDATE EXISTING NODE ---
            if editing_node_id not in nodes:
                 print(f"❌ Error: Node {editing_node_id} not found for update."); reset_node_form(); return

            print(f"Updating node '{nodes[editing_node_id]['name']}' ({editing_node_id})...")
            # Prevent type change during edit
            if nodes[editing_node_id]['type'] != node_type:
                 print(f"⚠️ Warning: Node type change during edit not supported. Keeping original type: {nodes[editing_node_id]['type']}.")
                 node_type = nodes[editing_node_id]['type'] # Revert type

            nodes[editing_node_id]['name'] = node_name
            nodes[editing_node_id]['options'] = options
            print(f"✅ Node '{node_name}' ({editing_node_id}) updated.")
            reset_node_form(clear_name=True)

        else: # --- ADD NEW NODE ---
            new_node_id = str(uuid.uuid4())[:8]
            while new_node_id in nodes: new_node_id = str(uuid.uuid4())[:8]

            nodes[new_node_id] = {
                'name': node_name, 'type': node_type, 'options': options, 'outputs': {}
            }
            print(f"✅ Node '{node_name}' ({new_node_id}) added.")
            reset_node_form(clear_name=True) # Clear form for next addition

    update_all_dropdowns()
    visualize_graph()

def on_load_node_clicked(b):
    """Loads the selected node's data into the form for editing."""
    global editing_node_id
    selected_id = select_node_edit_dropdown.value
    with edit_feedback_output:
        clear_output(wait=True)
        if not selected_id or selected_id not in nodes:
            print("❌ Please select a valid node to load."); reset_node_form(); return

        print(f"Loading node '{nodes[selected_id]['name']}' ({selected_id}) for editing...")
        editing_node_id = selected_id
        node_data = nodes[selected_id]

        # Populate form widgets
        node_name_text.value = node_data['name']
        node_type_dropdown.value = node_data['type']
        node_type_dropdown.disabled = True # Prevent type change during edit

        # Populate options
        options_accordion.selected_index = NODE_TYPE_TO_INDEX.get(node_data['type'], 0)

        if node_data['type'] == 'LLM Program':
            llm_ta = next((w for w in llm_options_box.children if isinstance(w, widgets.Textarea)), None)
            if llm_ta: llm_ta.value = node_data['options'].get('prompt', '')
        elif node_data['type'] == 'Show':
            show_ta = next((w for w in show_options_box.children if isinstance(w, widgets.Textarea)), None)
            if show_ta: show_ta.value = node_data['options'].get('text_template', '')
        elif node_data['type'] == 'Text Input': # Added option loading
            text_input_ta = next((w for w in text_input_options_box.children if isinstance(w, widgets.Textarea)), None)
            if text_input_ta: text_input_ta.value = node_data['options'].get('content', '')


        # Update button states
        add_update_node_button.description = 'Update Node'
        add_update_node_button.button_style = 'info'; add_update_node_button.icon = 'save'
        cancel_edit_button.visible = True

    visualize_graph() # Highlight the node being edited

def on_remove_node_clicked(b):
    """Removes the selected node and its connections."""
    global editing_node_id
    node_to_remove_id = select_node_edit_dropdown.value
    with edit_feedback_output:
        clear_output(wait=True)
        if not node_to_remove_id or node_to_remove_id not in nodes:
            print("❌ Please select a valid node to remove."); return

        removed_name = nodes[node_to_remove_id]['name']
        print(f"Removing node '{removed_name}' ({node_to_remove_id})...")
        del nodes[node_to_remove_id] # Delete the node

        # Remove incoming connections
        nodes_to_check = list(nodes.keys())
        for check_id in nodes_to_check:
             if check_id in nodes:
                outputs_to_check = list(nodes[check_id]['outputs'].items())
                for port_name, target_ids in outputs_to_check:
                    if isinstance(target_ids, str): target_ids = [target_ids] # Handle old format
                    if isinstance(target_ids, list) and node_to_remove_id in target_ids:
                        print(f"  Removing connection from {check_id} [{port_name}] -> {node_to_remove_id}")
                        nodes[check_id]['outputs'][port_name].remove(node_to_remove_id)
                        if not nodes[check_id]['outputs'][port_name]: del nodes[check_id]['outputs'][port_name]

        print(f"✅ Node '{removed_name}' and its connections removed.")
        if editing_node_id == node_to_remove_id:
            print("  Cancelling edit mode."); reset_node_form()
        else: visualize_graph() # Redraw if not cancelling

    update_all_dropdowns()


def on_add_connection_clicked(b):
    """Adds a connection (appends to list) between two nodes."""
    source_id = source_node_dropdown.value
    port_name = source_port_dropdown.value
    target_id = target_node_dropdown.value

    with add_connection_output:
        clear_output(wait=True)
        # Basic Validations
        if not all([source_id, port_name, target_id]): print("❌ Error: Select valid source, port, and target."); return
        if source_id == target_id: print("❌ Error: Cannot connect node to itself."); return
        if source_id not in nodes or target_id not in nodes: print("❌ Error: Source or target node missing."); return

        # Check port validity (redundant but safe)
        valid_ports = []; src_type = nodes[source_id]['type']
        if src_type == 'Button': valid_ports = ['on_click']
        elif src_type == 'LLM Program': valid_ports = ['result']
        elif src_type == 'Text Input': valid_ports = ['text_out'] # Added
        if port_name not in valid_ports: print(f"❌ Error: Port '{port_name}' invalid for type '{src_type}'."); return

        # Add Connection (Append Logic)
        output_dict = nodes[source_id]['outputs']
        if port_name not in output_dict: output_dict[port_name] = []
        if not isinstance(output_dict[port_name], list): output_dict[port_name] = [output_dict[port_name]] # Fix old format

        if target_id in output_dict[port_name]:
            print(f"ℹ️ Info: Connection already exists.")
        else:
            output_dict[port_name].append(target_id)
            print(f"✅ Connection added: {nodes[source_id]['name']} [{port_name}] -> {nodes[target_id]['name']}")

    visualize_graph() # Redraw graph

# --- Attach Handlers ---
# node_type_dropdown is observed earlier now
add_update_node_button.on_click(on_add_update_node_clicked)
cancel_edit_button.on_click(on_cancel_edit_clicked)
load_node_button.on_click(on_load_node_clicked)
remove_node_button.on_click(on_remove_node_clicked)
source_node_dropdown.observe(update_source_ports, names='value')
add_connection_button.on_click(on_add_connection_clicked)

# --- Layout the UI ---
edit_node_section = widgets.VBox([
    widgets.HTML("<b>Manage Nodes:</b>"),
    widgets.HBox([select_node_edit_dropdown, load_node_button, remove_node_button]),
    edit_feedback_output
])

add_node_section = widgets.VBox([
    widgets.HTML("<hr><b>Add/Update Node Details:</b>"),
    node_type_dropdown, node_name_text,
    widgets.HTML("<i>Node Specific Options:</i>"), options_accordion,
    widgets.HBox([add_update_node_button, cancel_edit_button]),
    add_node_output
])

add_connection_section = widgets.VBox([
    widgets.HTML("<hr><b>Add Connection:</b>"),
    source_node_dropdown, source_port_dropdown, target_node_dropdown,
    add_connection_button,
    add_connection_output
])

# --- Initial Population ---
update_all_dropdowns() # Call after everything is defined

# --- Display UI ---
display(widgets.VBox([
    edit_node_section,
    add_node_section,
    add_connection_section,
    widgets.HTML("<hr><b>Current Graph:</b>"),
    graph_output
]))

print("Node Definition UI updated with 'Text Input' type.")

In [ ]:
# @title Play Mode Execution Logic and UI (REVISED for Button Display)

import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import uuid
import traceback
import time # Added for placeholder delay if needed

# --- Global State Access ---
# Ensure these are accessible from previous cells
if 'nodes' not in globals(): nodes = {}
if 'client' not in globals(): client = None
# Define a placeholder if the function wasn't defined in cell 2 for some reason
if 'call_llm' not in globals():
    print("Warning: 'call_llm' function not found globally. Defining a placeholder.")
    def call_llm(prompt, model="gpt-3.5-turbo"):
        print(f"Placeholder LLM Call with prompt: {prompt[:100]}...")
        time.sleep(1) # Simulate API call delay
        return "Placeholder LLM response."


# --- Output Widgets ---
play_mode_button = widgets.Button(description="🚀 Enter Play Mode", button_style='warning')
play_output = widgets.Output()
full_log_output = widgets.Output()

# --- Execution Function (Revised LLM logic - No change from previous valid version) ---
active_executions = {}

def execute_node_flow(start_node_id, input_data=None):
    """Executes the node flow. LLM nodes now gather all connected Text Inputs."""
    queue = [(start_node_id, input_data)]
    run_id = str(uuid.uuid4())[:6]
    max_processed = len(nodes) * 10 # Increased safety break slightly
    processed_count = 0
    executed_in_run = set() # Track nodes executed in this specific run instance

    with full_log_output: print(f"\n--- Starting Execution Run {run_id} from Node {start_node_id} ---")

    while queue and processed_count < max_processed:
        current_node_id, current_input = queue.pop(0)
        processed_count += 1

        # Prevent re-executing the same node within this specific run instance
        # This helps prevent infinite loops caused by simple cycles triggering the same node repeatedly immediately
        if current_node_id in executed_in_run:
             with full_log_output: print(f"[{run_id}] Skipping node {current_node_id} - already processed in this run.")
             continue
        executed_in_run.add(current_node_id)


        if current_node_id not in nodes:
            with play_output: display(Markdown(f"⚠️ **Error:** Node `{current_node_id}` not found."))
            with full_log_output: print(f"[{run_id}] Error: Node {current_node_id} not found.")
            continue

        node_data = nodes[current_node_id]
        node_name = node_data['name']
        node_type = node_data['type']

        with full_log_output: print(f"[{run_id}] Executing Node: {node_name} ({current_node_id}), Type: {node_type}")

        output_data = None
        port_to_follow = None
        next_node_ids = []

        try:
            # --- Node Type Specific Logic ---
            if node_type == 'Button':
                output_data = current_input
                port_to_follow = 'on_click'

            elif node_type == 'Text Input':
                node_content = node_data['options'].get('content', '')
                output_data = {'node_name': node_name, 'text': node_content}
                port_to_follow = 'text_out'
                with full_log_output: print(f"[{run_id}] Text Input node produced output.")

            elif node_type == 'LLM Program': # --- REVISED LLM LOGIC ---
                llm_node_prompt_template = node_data['options'].get('prompt', '')
                final_prompt_parts = []
                gathered_texts = []

                with full_log_output: print(f"[{run_id}] LLM Node '{node_name}' scanning for connected Text Inputs...")
                for source_id, source_data in nodes.items():
                    if source_data['type'] == 'Text Input':
                        source_outputs = source_data.get('outputs', {})
                        text_out_targets = source_outputs.get('text_out', [])
                        if isinstance(text_out_targets, str): text_out_targets = [text_out_targets]
                        if current_node_id in text_out_targets:
                             text_content = source_data['options'].get('content', '')
                             text_node_name = source_data['name']
                             gathered_texts.append(f"Input from node '{text_node_name}':\n{text_content}")
                             with full_log_output: print(f"  > Found text from '{text_node_name}' ({source_id})")

                if gathered_texts:
                    context_block = "\n\n---\n".join(gathered_texts)
                    final_prompt_parts.append("--- Combined Text Inputs ---")
                    final_prompt_parts.append(context_block)
                    final_prompt_parts.append("---------------------------\n")
                else:
                     with full_log_output: print(f"[{run_id}] LLM node found no connected Text Input nodes.")
                     if isinstance(current_input, str) and current_input:
                         final_prompt_parts.append(f"Input (from previous node):\n{current_input}\n---")

                instruction = llm_node_prompt_template.replace('{input}', '').strip()
                if instruction: final_prompt_parts.append(f"User instruction: {instruction}")
                else: final_prompt_parts.append("User instruction: (No specific instruction provided)")

                final_prompt = "\n".join(final_prompt_parts)

                if client:
                    with full_log_output: print(f"[{run_id}] Calling LLM with combined prompt:\n{final_prompt[:500]}...")
                    output_data = call_llm(final_prompt) # Make sure call_llm is defined globally
                else:
                    output_data = "Error: OpenAI client not initialized."
                    with play_output: display(Markdown(f"⚠️ **LLM Error:** OpenAI client not initialized."))

                port_to_follow = 'result'

            elif node_type == 'Show':
                text_template = node_data['options'].get('text_template', 'No text configured.')
                display_input_str = ""
                if isinstance(current_input, str): display_input_str = current_input
                elif current_input is not None: display_input_str = str(current_input)
                display_text = text_template.replace('{input}', display_input_str)
                with play_output: display(Markdown(f"**{node_name}:**\n\n```\n{display_text}\n```"))
                output_data = display_text
                port_to_follow = None

            else:
                with play_output: display(Markdown(f"❓ **Unknown Node Type:** `{node_type}`"))
                with full_log_output: print(f"[{run_id}] Error: Unknown node type {node_type}")
                port_to_follow = None

            # --- Find Next Nodes ---
            if port_to_follow and port_to_follow in node_data.get('outputs', {}):
                potential_next_ids = node_data['outputs'][port_to_follow]
                if isinstance(potential_next_ids, str): next_node_ids = [potential_next_ids]
                elif isinstance(potential_next_ids, list): next_node_ids = potential_next_ids
                else: next_node_ids = []

        except Exception as e:
             # Print error to both outputs for visibility
             tb_str = traceback.format_exc()
             error_md = Markdown(f"💥 **Execution Error** in node `{node_name}` ({current_node_id}):\n```\n{e}\n{tb_str[:500]}...\n```")
             with play_output: display(error_md)
             with full_log_output:
                 print(f"[{run_id}] FATAL ERROR executing node {current_node_id}: {e}\n{tb_str}")
             continue # Stop this branch on error

        # --- Queue Next Nodes ---
        for next_id in next_node_ids:
            if next_id and next_id in nodes:
                with full_log_output: print(f"[{run_id}] Queueing next node: {next_id} from port '{port_to_follow}'")
                queue.append((next_id, output_data)) # Pass current node's output
            elif next_id:
                 with full_log_output: print(f"[{run_id}] Warning: Target node {next_id} not found.")


    if processed_count >= max_processed:
         with play_output: display(Markdown(f"⚠️ **Execution Limit Reached:** Processed {processed_count} nodes."))
         with full_log_output: print(f"[{run_id}] Execution limit ({max_processed}) reached.")


# --- Play Mode Button Handler (REVISED DISPLAY LOGIC) ---
def on_play_mode_clicked(b):
    # 1. Clear outputs and show initial messages
    with play_output:
        clear_output(wait=True)
        display(Markdown("--- **Play Mode Activated** ---"))
        print("Scanning for Button nodes...") # Goes into play_output widget
    with full_log_output:
        clear_output(wait=True)
        print("--- Play Mode Log ---") # Goes into full_log_output widget

    # 2. Find and prepare buttons WITHOUT displaying yet
    buttons_to_display = []
    found_buttons = False
    print(f"DEBUG: Nodes available: {list(nodes.keys())}") # Print available nodes to cell output

    for node_id, node_data in nodes.items():
        print(f"DEBUG: Checking node {node_id}, type: {node_data.get('type')}") # Check each node
        if node_data.get('type') == 'Button':
            print(f"DEBUG: Found Button node: {node_id}") # Confirm finding
            found_buttons = True
            try:
                button_widget = widgets.Button(
                    description=f"▶️ {node_data.get('name', 'Unnamed Button')}",
                    button_style='primary',
                    tooltip=f"Trigger node {node_id}"
                )

                # Define click handler specific to this button instance
                def handle_play_button_click(b_instance, n_id=node_id): # Capture node_id
                    with play_output: display(Markdown(f"▶️ *Clicked '{nodes[n_id]['name']}'...*"))
                    # Use try-except around flow execution for better error catching
                    try:
                        execute_node_flow(n_id, input_data=None) # Start the flow
                    except Exception as e_flow:
                         tb_flow = traceback.format_exc()
                         err_flow_md = Markdown(f"💥 **Flow Error** starting from `{nodes[n_id]['name']}` ({n_id}):\n```\n{e_flow}\n{tb_flow[:500]}...\n```")
                         with play_output: display(err_flow_md)
                         with full_log_output: print(f"FATAL ERROR starting flow from {n_id}: {e_flow}\n{tb_flow}")


                # Attach the handler
                button_widget.on_click(handle_play_button_click)
                # Add the fully prepared button to the list
                buttons_to_display.append(button_widget)
                print(f"DEBUG: Prepared button for {node_id}")

            except Exception as e_create:
                 # Catch errors during button creation/handler attachment
                 print(f"ERROR: Failed to create/prepare button for node {node_id}: {e_create}")
                 with play_output:
                     display(Markdown(f"⚠️ Error creating button for node `{node_data.get('name', node_id)}`"))


    # 3. Display results AFTER processing all nodes
    with play_output:
        if not found_buttons:
            print("No 'Button' nodes found in the graph.") # Print message if none found
            display(Markdown("No 'Button' nodes found in the graph.")) # Also display in widget
        else:
            print(f"Displaying {len(buttons_to_display)} buttons...") # Print confirmation
            for btn in buttons_to_display:
                display(btn) # Display each prepared button

play_mode_button.on_click(on_play_mode_clicked)

# Display UI Elements (Play button and output areas)
display(widgets.VBox([
    play_mode_button,
    widgets.HTML("<hr><b>Play Area:</b>"), play_output,
    widgets.HTML("<hr><b>Execution Log:</b>"), full_log_output
]))

print("Play Mode UI and Logic updated. Button display logic revised.")